# E. Explore Trial parameters, Output control and Noah-MP tables of SUMMA 3.0

## 1. Import pySUMMA 3.0.0

In [1]:
import pysumma as ps

## 2. Create Simulatioin Object

In [2]:
executable = "/usr/bin/summa.exe"
file_manager = 'SummaModel_ReynoldsAspenStand_StomatalResistance/settings/summa_fileManager_riparianAspenSimpleResistance.txt'

s = ps.Simulation(executable, file_manager)

## 3. Explore trial parameters file

The trial parameters file is a NetCDF file that specifies model parameters for GRUs and individual HRUs. This enables the user to overwrite the default and/or Noah-MP parameter values with local-specific ones. The trial parameters file is parsed by **`build/source/engine/read_param.f90:read_param()`**.

The trial parameters file contains a `gru` and/or an `hru` dimension, depending on whether GRU or HRU are being specified. The file can be used to overwrite any of the variables in the basin parameters file, in which the variable dimension should be `gru`, or in the local parameters file, in which the variable dimension should be `hru`. The file can contain zero or more parameter fields.

The file should include an index variable (`hruId` and/or `gruId`) that corresponds to the values used in the local attributes file to provide the mapping of parameter values to each individual HRU and GRU. Variable names in the trial parameters file must match those in **`build/source/dshare/var_lookup.f90:iLook_param`** for HRU parameters and **`build/source/dshare/var_lookup.f90:iLook_bpar`** for GRU parameters. Note that this matching is case-sensitive.

### 3.1 Show trial parameters using `print` command

In [3]:
# Read parameter trial netCDF file
print(s.trial_params)

<xarray.Dataset>
Dimensions:             (hru: 1)
Dimensions without coordinates: hru
Data variables:
    hruIndex            (hru) float64 1.001e+03
    frozenPrecipMultip  (hru) float64 1.0
    theta_mp            (hru) float64 0.2211
    theta_sat           (hru) float64 0.4792
    theta_res           (hru) float64 0.1
    vGn_alpha           (hru) float64 -0.858
    vGn_n               (hru) float64 1.306
    f_impede            (hru) float64 0.0
    k_soil              (hru) float64 3.53e-06
    k_macropore         (hru) float64 0.06255
    critSoilWilting     (hru) float64 0.125
    critSoilTranspire   (hru) float64 0.2
    winterSAI           (hru) float64 0.45
    summerLAI           (hru) float64 2.5
    heightCanopyTop     (hru) float64 9.5
    heightCanopyBottom  (hru) float64 3.0
    kAnisotropic        (hru) float64 1.0
    rootDistExp         (hru) float64 1.0
    zScale_TOPMODEL     (hru) float64 4.057
    qSurfScale          (hru) float64 84.06
Attributes:
    history: 

### 3.2 Change value of  `summerLAI` in trial parameters

In [4]:
s.assign_trial_params('summerLAI', float(2.0))
print(s.trial_params)

<xarray.Dataset>
Dimensions:             (hru: 1)
Dimensions without coordinates: hru
Data variables:
    hruIndex            (hru) float64 1.001e+03
    frozenPrecipMultip  (hru) float64 1.0
    theta_mp            (hru) float64 0.2211
    theta_sat           (hru) float64 0.4792
    theta_res           (hru) float64 0.1
    vGn_alpha           (hru) float64 -0.858
    vGn_n               (hru) float64 1.306
    f_impede            (hru) float64 0.0
    k_soil              (hru) float64 3.53e-06
    k_macropore         (hru) float64 0.06255
    critSoilWilting     (hru) float64 0.125
    critSoilTranspire   (hru) float64 0.2
    winterSAI           (hru) float64 0.45
    summerLAI           (hru) float64 2.0
    heightCanopyTop     (hru) float64 9.5
    heightCanopyBottom  (hru) float64 3.0
    kAnisotropic        (hru) float64 1.0
    rootDistExp         (hru) float64 1.0
    zScale_TOPMODEL     (hru) float64 4.057
    qSurfScale          (hru) float64 84.06
Attributes:
    history: 

## 4. Explore Output control text file

The output control file is an ASCII file that specifies which variables are retained in the SUMMA output files. The output control file is parsed by **`build/source/dshare/popMetadat.f90:read_output_file()`**

SUMMA is pretty flexible in its output. There are many variables that you can output and for most of them you can also choose to record summary statistics. For example, you can configure the model to run with meteorological forcings that are defined every hour, but only save summary output with a daily time step. This flexibility comes at the small price that you need to be clear in specifying what output you want.

The output control file includes a listing of model variables that you would like to store, with one model variable per line. The variables that are available for output are the individual entries in the data structures specified in **`build/source/dshare/var_lookup.f90`**. Because there are many, there is not much point in repeating them here, but we direct the user to the model code. Any of the variables specified in the following structures can be specified in the output control file: **`iLook_time`, `iLook_force`, `iLook_attr`, `iLook_type`, `iLook_param`, `iLook_index`, `iLook_prog`, `iLook_diag`, `iLook_flux`, `iLook_bpar`, `iLook_bvar`, `iLook_deriv`**. SUMMA will print an error message if a specific variable cannot be output, so the faster way may be to select any variable in **`build/source/dshare/var_lookup.f90`** and remove it if it is not available for output. In addition, some of the variables may only be useful for debugging use, but that is up to the user.

At a minimum, each line in the output control file will contain two fields, separated by a **`|`**. The first field will be the variable name as specified in **`build/source/dshare/var_lookup.f90`** (case-sensitive). The second field will be the frequency of the model output specified as a multiple of the time resolution in the model forcing files. Thus, if you want to output data for every forcing time step, then this value should be equal to 1. If you want daily output and your forcing frequency is 3 hours, then this value should be equal to 8. Note that you can specify different output frequencies for separate variables, but at this time you can specify only a single output frequency for each variable. For example, you can store **`scalarSenHeatTotal`** with an output frequency of 1 and **`scalarLatHeatTotal`** with an output frequency of 8, but you cannot specify two different output frequencies for **`scalarSenHeatTotal`**.

For most variables you can also output a statistical summary if you output variables at a lower frequency than your forcing frequency. To do this, you extend the number of fields you specify in the output control file, with all fields separated by a **`|`**. For the fields after the first two, you specify a series of 0's and 1's, which indicate that a specific statistic should not (0) or should be stored (1). The available statistics are (in order) the instantaneous value, the sum over the interval, the mean, the variance, the minimum, the maximum and the mode. So, a complete line in the output control file would be

```
| varName            | outFreq | inst | sum | mean | var | min | max | mode
| scalarSenHeatTotal | 24      | 0    | 1   | 1    | 0   | 1   | 1   | 0
```

In this example, the first line is a comment (starts with `!`) and then the sum, mean, min, max are calculated for **`scalarSenHeatTotal`** across 24 forcing time steps and written to the output file.

<br>

### Modifying output
And one more, we can also modify what get's written to output. 
The output control file represents the options available through columns of numeric values.
These numbers represent how to write the output. 
From the SUMMA documentation they are arranged as:

```
! varName          | outFreq | inst | sum | mean | var | min | max | mode
```

As before, let's look at what's in the `output_control` by simply printing it out.

<br>

### 4.1 Show output control file using `print` command

In [5]:
print(s.output_control)

nSnow                                | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
nSoil                                | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
pptrate                              | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
airtemp                              | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarRainPlusMelt                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarSWE                            | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarThroughfallSnow                | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarThroughfallRain                | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarSnowSublimation                | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarInfiltration                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarExfiltration                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarSurfaceRunoff                  | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarSurfaceTemp                    | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarSenHeatTotal                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
scalarLatHeatTotal                

### 4.2 Show value of  `scalarLatHeatTotal` in output control text file

In [6]:
print(s.output_control['scalarLatHeatTotal'])

scalarLatHeatTotal                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0


### 4.3 Change value of  `scalarLatHeatTotal` in output control text file

In [7]:
# Change the output statistic from hourly instantaneous to daily total
s.output_control['scalarLatHeatTotal'] = [24, 1, 0, 0, 0, 0, 0, 0]
print(s.output_control['scalarLatHeatTotal'])

scalarLatHeatTotal                   | 24 | 1 | 0 | 0 | 0 | 0 | 0 | 0


### 4.4 Change value of  `scalarLatHeatTotal`  back in output control text file

In [8]:
s.output_control['scalarLatHeatTotal'] = [1, 0, 1, 0, 0, 0, 0, 0]
print(s.output_control['scalarLatHeatTotal'])

scalarLatHeatTotal                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0


## 5. Explore Noah-MP tables text file

SUMMA uses some of the input files and routines from the Noah-MP Land Surface Model (LSM). These routines are mostly contained in the **`build/source/noah-mp`** directory, although a few can be found elsewhere in the code as well. For example, the code that parses the Noah-MP tables is in **`build/source/driver/multi_driver.f90:SOIL_VEG_GEN_PARM()`**. The names of these tables is hard-wired (since that it is how this is implemented in Noah-MP). The file path for these tables is constructed as **`<SETNGS_PATH>/<NOAH_TABLE>`**, where **`SETNGS_PATH`** is defined in the master configuration file and **`NOAH_TABLE`** is **`VEGPARM.TBL`**, **`SOILPARM.TBL`** and **`GENPARM.TBL`**. The format for these files remains unchanged from their specification in Noah-MP, see here for an example. The parameter values in the Noah-MP overwrite the default values that have already been specified based on soil and vegetation type.

In [9]:
for genparm in s.genparm:
    print(genparm.splitlines()[0])

General Parameters
SLOPE_DATA
9
0.1 
0.6
1.0
0.35
0.55
0.8
0.63
0.0
0.0
SBETA_DATA
-2.0
FXEXP_DATA
2.0
CSOIL_DATA
2.00E+6
SALP_DATA
2.6
REFDK_DATA
2.0E-6
REFKDT_DATA
3.0
FRZK_DATA
0.15
ZBOT_DATA
-8.0
CZIL_DATA
0.1
SMLOW_DATA
0.5
SMHIGH_DATA
3.0
LVCOEF_DATA
0.5


In [10]:
for mptable in s.mptable:
    print(mptable.splitlines()[0])

&noah_mp_usgs_veg_categories
 VEG_DATASET_DESCRIPTION = "USGS"
 NVEG = 27
/
&noah_mp_usgs_parameters
 ! NVEG = 27
 !  1: Urban and Built-Up Land
 !  2: Dryland Cropland and Pasture
 !  3: Irrigated Cropland and Pasture
 !  4: Mixed Dryland/Irrigated Cropland and Pasture
 !  5: Cropland/Grassland Mosaic
 !  6: Cropland/Woodland Mosaic
 !  7: Grassland
 !  8: Shrubland
 !  9: Mixed Shrubland/Grassland
 ! 10: Savanna
 ! 11: Deciduous Broadleaf Forest
 ! 12: Deciduous Needleleaf Forest
 ! 13: Evergreen Broadleaf Forest
 ! 14: Evergreen Needleleaf Forest
 ! 15: Mixed Forest
 ! 16: Water Bodies
 ! 17: Herbaceous Wetland
 ! 18: Wooded Wetland
 ! 19: Barren or Sparsely Vegetated
 ! 20: Herbaceous Tundra
 ! 21: Wooded Tundra
 ! 22: Mixed Tundra
 ! 23: Bare Ground Tundra
 ! 24: Snow or Ice
 ! 25: Playa
 ! 26: Lava
 ! 27: White Sand

 ISURBAN   = 1
 ISWATER   = 16
 ISBARREN  = 19
 ISSNOW    = 24
 EBLFOREST = 13

 !-----------------------------------------------------------------------------------

In [11]:
for soilparm in s.soilparm:
    print(soilparm.splitlines()[0])

Soil Parameters
STAS
19,1   'BB      DRYSMC      F11     MAXSMC   REFSMC   SATPSI  SATDK       SATDW     WLTSMC  QTZ    '
1,     2.79,    0.010,    -0.472,   0.339,   0.236,   0.069,  1.07E-6,  0.608E-6,   0.010,  0.92, 'SAND'
2,     4.26,    0.028,    -1.044,   0.421,   0.383,   0.036,  1.41E-5,  0.514E-5,   0.028,  0.82, 'LOAMY SAND'
3,     4.74,    0.047,    -0.569,   0.434,   0.383,   0.141,  5.23E-6,  0.805E-5,   0.047,  0.60, 'SANDY LOAM'
4,     5.33,    0.084,     0.162,   0.476,   0.360,   0.759,  2.81E-6,  0.239E-4,   0.084,  0.25, 'SILT LOAM'
5,     5.33,    0.084,     0.162,   0.476,   0.383,   0.759,  2.81E-6,  0.239E-4,   0.084,  0.10, 'SILT'
6,     5.25,    0.066,    -0.327,   0.439,   0.329,   0.355,  3.38E-6,  0.143E-4,   0.066,  0.40, 'LOAM'
7,     6.66,    0.067,    -1.491,   0.404,   0.314,   0.135,  4.45E-6,  0.990E-5,   0.067,  0.60, 'SANDY CLAY LOAM'
8,     8.72,    0.120,    -1.118,   0.464,   0.387,   0.617,  2.04E-6,  0.237E-4,   0.120,  0.10, 'SILTY CLAY LOAM'

In [12]:
for vegparm in s.vegparm:
    print(vegparm.splitlines()[0])

Vegetation Parameters
USGS
27,1, 'SHDFAC NROOT   RS      RGL      HS      SNUP  MAXALB   LAIMIN  LAIMAX  EMISSMIN EMISSMAX ALBEDOMIN ALBEDOMAX  Z0MIN    Z0MAX  '
1,      .10,   1,    200.,   999.,   999.0,   0.04,    46.,    1.00,   1.00,   .880,    .880,     .15,      .15,      .50,     .50,     'Urban and Built-Up Land'  
2,      .80,   3,     40.,   100.,   36.25,   0.04,    66.,    1.56,   5.68,   .920,    .985,     .17,      .23,      .05,     .15,     'Dryland Cropland and Pasture' 
3,      .80,   3,     40.,   100.,   36.25,   0.04,    66.,    1.56,   5.68,   .930,    .985,     .20,      .25,      .02,     .10,     'Irrigated Cropland and Pasture' 
4,      .80,   3,     40.,   100.,   36.25,   0.04,    66.,    1.00,   4.50,   .920,    .985,     .18,      .23,      .05,     .15,     'Mixed Dryland/Irrigated Cropland and Pasture' 
5,      .80,   3,     40.,   100.,   36.25,   0.04,    68.,    2.29,   4.29,   .920,    .980,     .18,      .23,      .05,     .14,     'Cropland/Grassl